In [7]:
import pandas as pd 
from pulp import *
import re

In [24]:
with open('data.txt', 'r') as fp:
    x = fp.readlines()
x = [re.sub('\n','',i) for i in x]

In [27]:
df = pd.DataFrame([tuple(re.split(',',i)) for i in x])
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.reset_index(drop=True, inplace=T)

,Plant,Inventory Cost(100 units),Vendor,Capacity,Disruption Probability(1 - µ),TTR of vendor
1,1.ABX,31,1,1.2,0.0667,1
2,2.ABN,30,1,1.2,0.0667,1
3,3.GHY,32,2,0.9,0.1333,1.2
4,4.KJU,29,3,0.5,0.2000,1.4
5,5.PFR,30,4,0.5,0.1667,1.6
6,6.JKI1,30,5,2.4,0.0667,1.8
7,7.JKI2,33,5,2.4,0.0667,1.8
8,8.JKI3,31,5,2.4,0.0667,1.8
9,9.CFG1,500,6,0.2,0.0667,2
10,10.CFG2,550,7,0.4,0.1667,2.2


In [45]:
# Create a list of the product items
node_1 = ['steel', 'copper', 'rubber']
node_2 = ['copper', 'steel', 'rubber']
demand = {'steel':2700, 'copper':107, 'rubber':203}

In [46]:
f = {'steel':2.7, 'copper':1.7, 'rubber':2.3}

In [61]:
prob = LpProblem("Disruption_Risk_Mitigation",LpMinimize)

In [62]:
mat_vars = LpVariable.dicts("Material_kg",
                                     ((i, j) for i in node_1 for j in node_2),
                                     lowBound=0,
                                     cat='Continuous')

In [63]:
goods_prod = LpVariable.dict("good_prod",node_1,lowBound=0,cat='Continuous')

In [64]:
lost_sales = LpVariable.dict("lost_sales",node_1,lowBound=0,cat='Continuous')

In [65]:
lost_sales

{'steel': lost_sales_steel,
 'copper': lost_sales_copper,
 'rubber': lost_sales_rubber}

In [66]:
prob += lpSum([f[i]*lost_sales[i] for i in node_1])

In [67]:
for n in node_1:
    for m in node_2:
        prob += lpSum(mat_vars[(n, m)]) + lost_sales[n] == demand[n]

In [68]:
for n in node_1:
    for m in node_2:
        prob += lpSum(mat_vars[(n, m)]) + lost_sales[n] == demand[n]

In [69]:
prob.solve()

1

In [70]:
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Material_kg_('copper',_'copper') = 107.0
Material_kg_('copper',_'rubber') = 107.0
Material_kg_('copper',_'steel') = 107.0
Material_kg_('rubber',_'copper') = 203.0
Material_kg_('rubber',_'rubber') = 203.0
Material_kg_('rubber',_'steel') = 203.0
Material_kg_('steel',_'copper') = 2700.0
Material_kg_('steel',_'rubber') = 2700.0
Material_kg_('steel',_'steel') = 2700.0


In [71]:
value(prob.objective)

0.0

In [72]:
[f[i]*lost_sales[i] for i in node_1]

[2.7*lost_sales_steel + 0.0,
 1.7*lost_sales_copper + 0.0,
 2.3*lost_sales_rubber + 0.0]